# Day 3 - Conversational AI - aka Chatbot!

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-BMuGj
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyDW


In [3]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'
docker_server_name="0.0.0.0"

import time
import signal
import os
import psutil

def kill_process_on_port(port):
    for proc in psutil.process_iter(['pid', 'name', 'connections']):
        try:
            for conn in proc.info['connections']:
                if conn.laddr.port == port:
                    os.kill(proc.info['pid'], signal.SIGTERM)
                    time.sleep(1)  # Give it time to terminate
                    try:
                        os.kill(proc.info['pid'], signal.SIGKILL)  # Force kill if still running
                    except:
                        pass
        except:
            continue

In [4]:
system_message = "You are a helpful assistant"

# Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [5]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## And then enter Gradio's magic!

In [ ]:
import time

# Close existing instances
gr.close_all()
# Add a small delay to ensure port is released
time.sleep(2)

gr.ChatInterface(fn=chat, type="messages").launch(server_name=docker_server_name, server_port=7860)

History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'i want to run fast'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'i want to run fast', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Improving your running speed involves a combination of proper training, nutrition, and recovery. Here are some tips to help you run faster:\n\n### Training Tips:\n\n1. **Interval Training**: Incorporate high-intensity interval training (HIIT) into your routine. This involves alternating between short bursts of high-speed running and recovery periods.\n\n2. **Tempo Runs**: These runs help improve your lactate threshold. Run at a pace that is comfortably hard for a sustained period (20-30 minutes) after a warm-up.\n\n3. **Hill Workouts**: Running uphill increases strength and power in your legs, which can enhance speed. Find a hill and do repeats, running up at a hard pace and jogging or walking 

In [4]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [6]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [7]:
gr.close_all()
gr.ChatInterface(fn=chat, type="messages").launch(server_name=docker_server_name, server_port=7860)

* Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


In [7]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [9]:
gr.close_all()
gr.ChatInterface(fn=chat, type="messages").launch(server_name=docker_server_name, server_port=7860)

OSError: Cannot find empty port in range: 7860-7860. You can specify a different port by setting the GRADIO_SERVER_PORT environment variable or passing the `server_port` parameter to `launch()`.

In [2]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
# First close gracefully
gr.close_all()
time.sleep(2)

# Then forcefully kill anything on port 7860
kill_process_on_port(7860)
'''
# Find Python processes
ps aux | grep python

# Kill any existing Gradio processes
pkill -f gradio
'''
time.sleep(2)
gr.ChatInterface(fn=chat, type="messages").launch(server_name=docker_server_name, server_port=7860)

* Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


: 

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>